In [59]:
from faker import Faker
import re
import random

fake = Faker()

def take_initial(s):
    initials = [i[0] for i in s.split()]
    while len(initials) < 2:
        initials.append(fake.random_uppercase_letter())
    return ''.join(initials).upper()

def wrap_single_quote(s):
    return "'" + s + "'"

# compartment
FN = 'FN'
AD = 'AD'
OP = 'OP'

# security level
SL_P = 'P'
SL_C = 'C'
SL_S = 'S'
SL_HS = 'HS'

# group 
UNIV = 'UNIV'

# ols column name
ols = 'ols_pol_col'

def get_school_group(school_id):
    return 's_{}'.format(school_id)

def get_course_schedule_group(school_id, cs_id):
    return 's_{}_{}'.format(school_id, cs_id)

def get_security_label(security_level, compartments, groups):
    g = ','.join(groups)
    c = ','.join(compartments)
    return "CHAR_TO_LABEL('ols_pol','{}:{}:{}')".format(security_level, c, g)

In [60]:
school_num = 2
school_security_level = SL_P
school_id = [i+1 for i in range(school_num)]
school_name = ['School of ' + fake.job().split()[0] + ''.join(fake.random_letters(length=3)) for i in range(school_num)]
school_email = [ take_initial(school_name[i]) + str(i) + '@nus.edu.sg' for i in range(school_num)]
school_address = [school_name[i] + ', ' + fake.address().replace('\n', '') for i in range(school_num)]
school_postal = [fake.postalcode() for i in range(school_num)]
school_tel = [fake.phone_number() for i in range(school_num)]
school_label = ['SOC', 'BIZ']

school_insert_sql = 'INSERT ALL\n'+ '\n'.join(
    ['INTO school (id, name, email, address, postal, tel)\
    VALUES ({},{},{},{},{},{})'.format(
    school_id[i],
    wrap_single_quote(school_name[i]),
    wrap_single_quote(school_email[i]),
    wrap_single_quote(school_address[i]),
    wrap_single_quote(school_postal[i]),
    wrap_single_quote(school_tel[i])) for i in range(school_num)]) + '\n SELECT * FROM dual;\n\n'

# print(school_insert_sql)

In [61]:
academic_num = 4
asy_slevel = SL_P
asy_id = [i+1 for i in range(academic_num)]
academic_start_year = [str(2017-academic_num//2+i) for i in range(academic_num)]
academic_end_year = [str(2018-academic_num//2+i) for i in range(academic_num)]
asy_sl = [get_security_label(asy_slevel, [],[]) for i in asy_id]

academic_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ["INTO academic_year (id, start_year, end_year)\
    VALUES ({},{},{})".format(
    asy_id[i],
    wrap_single_quote(academic_start_year[i]),
    wrap_single_quote(academic_end_year[i])) for i in range(academic_num)]) + '\n SELECT * FROM dual;\n\n'

# print(academic_insert_sql)

In [62]:
semster_num = academic_num * 2
semster_start_end_dates = [('-08-01 00:00:00 +08:00','-12-31 00:00:00 +08:00'), ('-01-01 00:00:00 +08:00','-05-31 00:00:00 +08:00')]
sem_id = [i+1 for  i in range(semster_num)]
sem_slevel = SL_P
semster = [i%2+1 for i in range(semster_num)]
semster_start_time = [str(2017-semster_num//4+(i+1)//2)+semster_start_end_dates[i%2][0] for i in range(semster_num)]
semster_end_time = [str(2017-semster_num//4+(i+1)//2)+semster_start_end_dates[i%2][1] for i in range(semster_num)]
sem_sl = [get_security_label(sem_slevel, [],[]) for i in range(semster_num)]
semster_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO semester (id, semester, start_date, end_date, academic_year_id)\
    VALUES({}, {}, TIMESTAMP {},TIMESTAMP {}, {})'.format(
    sem_id[i],
    str(semster[i]),
    wrap_single_quote(semster_start_time[i]),
    wrap_single_quote(semster_end_time[i]),
    str(i//2+1)) for i in range(semster_num)]) + '\n SELECT * FROM dual;\n\n'

# print(semster_insert_sql)

In [63]:
admin_officer_num = 2 * school_num

admin_officer_id = [ fake.uuid4() for i in range(admin_officer_num)]
admin_officer_name = [ fake.name() for i in range(admin_officer_num)]
admin_officer_office_email = [ admin_officer_name[i].replace(" ", ".") + str(i) + '@nus.edu.sg' for i in range(admin_officer_num)]
admin_officer_office_address = [ fake.address().replace('\n', '') for i in range(admin_officer_num)]
admin_officer_office_postal = [ fake.postalcode() for i in range(admin_officer_num)]
admin_officer_office_tel = [ fake.phone_number() for i in range(admin_officer_num)]
admin_officer_school_id = [ i%school_num+1 for i in range(admin_officer_num)]

admin_officer_insert_sql = 'INSERT ALL\n' + '\n'.join(['INTO admin_officer (id, first_name, last_name, gender, email, office_address, office_postal, office_tel, school_id) VALUES({},{},{},{},{},{},{},{},{})'.format(
    wrap_single_quote(admin_officer_id[i]),
    wrap_single_quote(admin_officer_name[i].split(' ')[0]),
    wrap_single_quote(admin_officer_name[i].split(' ')[1]),
    wrap_single_quote(random.choice(['F', 'M'])),
    wrap_single_quote(admin_officer_office_email[i]),
    wrap_single_quote(admin_officer_office_address[i]),
    wrap_single_quote(admin_officer_office_postal[i]),
    wrap_single_quote(admin_officer_office_tel[i]),
    str(admin_officer_school_id[i])) for i in range(admin_officer_num)]) + '\n SELECT * FROM dual;\n\n'

In [64]:
professor_num = 2 * school_num
professor_id = [ fake.uuid4() for i in range(professor_num)]
professor_name = [ fake.name() for i in range(professor_num)]
professor_office_email = [ professor_name[i].replace(" ", ".") + str(i) + '@nus.edu.sg' for i in range(professor_num)]
professor_office_address = [ fake.address().replace('\n', '') for i in range(professor_num)]
professor_office_postal = [ fake.postalcode() for i in range(professor_num)]
professor_office_tel = [ fake.phone_number() for i in range(professor_num)]
professor_school_id = [ i%school_num+1 for i in range(professor_num)]

professor_insert_sql = 'INSERT ALL\n' + '\n'.join(['INTO professor (id, first_name, last_name, gender, email, office_address, office_postal, office_tel, school_id) VALUES({},{},{},{},{},{},{},{},{})'.format(
    wrap_single_quote(professor_id[i]),
    wrap_single_quote(professor_name[i].split(' ')[0]),
    wrap_single_quote(professor_name[i].split(' ')[1]),
    wrap_single_quote(random.choice(['F', 'M'])),
    wrap_single_quote(professor_office_email[i]),
    wrap_single_quote(professor_office_address[i]),
    wrap_single_quote(professor_office_postal[i]),
    wrap_single_quote(professor_office_tel[i]),
    str(professor_school_id[i])) for i in range(professor_num)]) + '\n SELECT * FROM dual;\n\n'

In [65]:
student_num = 2 * school_num
stu_slevel = SL_C
student_id = [ fake.uuid4() for i in range(student_num)]
student_name = [ fake.name() for i in range(student_num)]
student_email = [ student_name[i].replace(" ", ".") + str(i) + '@nus.edu.sg' for i in range(student_num)]
student_address = [ fake.address().replace('\n', '') for i in range(student_num)]
student_postal = [ fake.postalcode() for i in range(student_num)]
student_tel = [ fake.phone_number() for i in range(student_num)]
stu_sch_id = [ i%school_num+1 for i in range(student_num)]

In [66]:
loan_num = student_num // 2
loan_slevel = SL_S
loan_id = [i+1 for i in range(loan_num)]
loan_amount = [ random.randint(10000,50000) for i in range(loan_num)]
loan_start_date = [ fake.past_date().replace().strftime("%Y-%m-%d %H:%M:%S") + ' +08:00' for i in range(loan_num)]
loan_due_date = [ fake.future_date(end_date='+365d').strftime("%Y-%m-%d %H:%M:%S") + ' +08:00' for i in range(loan_num)]
loan_amount_paid = [ "{0:.2f}".format(random.random() * loan_amount[i]) for i in range(loan_num)]
loan_student_id = [ student_id[i*2] for i in range(loan_num)]
loan_sl = [get_security_label(loan_slevel, [FN], [school_label[i%2]]) for i in range(loan_num)]

loan_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO loan (id, amount, start_date, due_date, paid_amount, student_id, {})\
    VALUES({},{},TIMESTAMP {},TIMESTAMP {},{},{},{})'.format(
    ols,
    loan_id[i],
    str(loan_amount[i]),
    wrap_single_quote(loan_start_date[i]),
    wrap_single_quote(loan_due_date[i]),
    str(loan_amount_paid[i]),
    wrap_single_quote(loan_student_id[i]),
    loan_sl[i]) for i in range(loan_num)]) + '\n SELECT * FROM dual;\n\n'

# print(loan_insert_sql)

In [67]:
course_id = ['CS5228', 'BT4211']
course_num = len(course_id)
course_index = [ fake.random_number(digits=4) for i in range(course_num)]
course_name = [ fake.job()[:50] for i in range(course_num)]
course_description = [ fake.sentence() for i in range(course_num)]
course_school_id = [1,2]
course_label = course_id[:]
course_insert_sql = 'INSERT ALL\n' + '\n'.join(['INTO course (id, idx, name, description, school_id) VALUES({},{},{},{},{})'.format(
    i+1,
    str(course_index[i]),
    wrap_single_quote(course_name[i]),
    wrap_single_quote(course_description[i]),
    str(course_school_id[i])) for i in range(course_num)]) + '\n SELECT * FROM dual;\n\n'

print(course_insert_sql)

INSERT ALL
INTO course (id, idx, name, description, school_id) VALUES(1,7711,'Nature conservation officer','However sport spend month late particular why.',1)
INTO course (id, idx, name, description, school_id) VALUES(2,9797,'Hotel manager','Owner action no.',2)
 SELECT * FROM dual;




In [68]:
course_schedule_num = course_num
cs_id = [i+1 for i in range(course_schedule_num)]
course_schedule_course_id = [ i%course_num+1 for i in range(course_schedule_num)] 
course_schedule_semster_id = [ i%semster_num+1 for i in range(course_schedule_num)]
course_schedule_professor_id = [ professor_id[i%professor_num] for i in range(course_schedule_num)]

course_schedule_insert_sql = 'INSERT ALL\n' + '\n'.join(['INTO course_schedule (id, course_id, semester_id, professor_id) VALUES({},{},{},{})'.format(
    i+1,
    str(course_schedule_course_id[i]),
    str(course_schedule_semster_id[i]),
    wrap_single_quote(course_schedule_professor_id[i])) for i in range(course_schedule_num)]) + '\n SELECT * FROM dual;\n\n'

print(course_insert_sql)

INSERT ALL
INTO course (id, idx, name, description, school_id) VALUES(1,7711,'Nature conservation officer','However sport spend month late particular why.',1)
INTO course (id, idx, name, description, school_id) VALUES(2,9797,'Hotel manager','Owner action no.',2)
 SELECT * FROM dual;




In [69]:
course_enrollment_num = student_num
ce_slevel = SL_C
ce_id = [i+1 for i in range(course_enrollment_num)]
ce_stu_id = [ student_id[i] for i in range(course_enrollment_num)]
ce_cs_id = [ cs_id[i//3] for i in range(course_enrollment_num)]
ce_sl = [get_security_label(ce_slevel, [OP], [course_label[i//2]]) for i in range(course_enrollment_num)]

course_enrollment_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO course_enrollment(id, student_id,course_schedule_id,{})\
    VALUES({},{},{},{})'.format(
    ols,
    str(ce_id[i]),
    wrap_single_quote(ce_stu_id[i]),
    str(ce_cs_id[i]),
    ce_sl[i]) for i in range(course_enrollment_num)]) + '\n SELECT * FROM dual;\n\n'

print(course_enrollment_insert_sql)

INSERT ALL
INTO course_enrollment(id, student_id,course_schedule_id,ols_pol_col)    VALUES(1,'68abf025-9639-1a19-917b-3b04139be79e',1,CHAR_TO_LABEL('ols_pol','C:OP:CS5228'))
INTO course_enrollment(id, student_id,course_schedule_id,ols_pol_col)    VALUES(2,'0bf30fe0-4474-563f-7622-b7d0a8d2389f',1,CHAR_TO_LABEL('ols_pol','C:OP:CS5228'))
INTO course_enrollment(id, student_id,course_schedule_id,ols_pol_col)    VALUES(3,'aa92e04a-b38b-c6c8-e614-7fd9b23b749a',1,CHAR_TO_LABEL('ols_pol','C:OP:BT4211'))
INTO course_enrollment(id, student_id,course_schedule_id,ols_pol_col)    VALUES(4,'cb832cc4-32c5-469a-46ea-625b4adbb0ba',2,CHAR_TO_LABEL('ols_pol','C:OP:BT4211'))
 SELECT * FROM dual;




In [70]:
stu_group_label = [get_security_label(stu_slevel, [], [course_label[i//3]]) for i in range(student_num)]

student_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO student (id, first_name, last_name, gender, matric_no, email, address, postal, tel, school_id, {}) \
    VALUES({},{},{},{},{},{},{},{},{},{},{})'.format(
    ols,
    wrap_single_quote(student_id[i]),
    wrap_single_quote(student_name[i].split(' ')[0]),
    wrap_single_quote(student_name[i].split(' ')[1]),
    wrap_single_quote(random.choice(['F', 'M'])),
    wrap_single_quote('AW' + student_id[i][:18]),
    wrap_single_quote(student_email[i]),
    wrap_single_quote(student_address[i]),
    wrap_single_quote(student_postal[i]),
    wrap_single_quote(student_tel[i]),
    str(stu_sch_id[i]), 
    stu_group_label[i]) for i in range(student_num)]) + '\n SELECT * FROM dual;\n\n'

print(student_insert_sql)

INSERT ALL
INTO student (id, first_name, last_name, gender, matric_no, email, address, postal, tel, school_id, ols_pol_col)     VALUES('68abf025-9639-1a19-917b-3b04139be79e','Andrew','Adams','M','AW68abf025-9639-1a19','Andrew.Adams0@nus.edu.sg','076 Wilson ShoresCodyborough, GA 71719','25756','1-902-760-7970x972',1,CHAR_TO_LABEL('ols_pol','C::CS5228'))
INTO student (id, first_name, last_name, gender, matric_no, email, address, postal, tel, school_id, ols_pol_col)     VALUES('0bf30fe0-4474-563f-7622-b7d0a8d2389f','Philip','Atkins','F','AW0bf30fe0-4474-563f','Philip.Atkins1@nus.edu.sg','848 Lambert CentersNorth Angelamouth, IL 83612','99899','1-712-995-2716',2,CHAR_TO_LABEL('ols_pol','C::CS5228'))
INTO student (id, first_name, last_name, gender, matric_no, email, address, postal, tel, school_id, ols_pol_col)     VALUES('aa92e04a-b38b-c6c8-e614-7fd9b23b749a','Mrs.','Lorraine','F','AWaa92e04a-b38b-c6c8','Mrs..Lorraine.Avery2@nus.edu.sg','USNV MunozFPO AA 41592','38546','973.696.1667',1,CHA

In [71]:
grade_num = course_enrollment_num
grade_slevel = SL_C
grade_ce_id = ce_id[:]
grades = [fake.random_number(digits=2) for i in range(grade_num)]
grades_sl = [get_security_label(ce_slevel, [AD], [course_label[i//2]]) for i in range(grade_num)]

grade_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO grade(course_enrollment_id, grade, {})\
    VALUES({},{},{})'.format(
    ols,
    str(grade_ce_id[i]),
    str(grades[i]),
    grades_sl[i]) for i in range(course_enrollment_num)]) + '\n SELECT * FROM dual;\n\n'

In [72]:
print(grade_insert_sql)

INSERT ALL
INTO grade(course_enrollment_id, grade, ols_pol_col)    VALUES(1,8,CHAR_TO_LABEL('ols_pol','C:AD:CS5228'))
INTO grade(course_enrollment_id, grade, ols_pol_col)    VALUES(2,42,CHAR_TO_LABEL('ols_pol','C:AD:CS5228'))
INTO grade(course_enrollment_id, grade, ols_pol_col)    VALUES(3,38,CHAR_TO_LABEL('ols_pol','C:AD:BT4211'))
INTO grade(course_enrollment_id, grade, ols_pol_col)    VALUES(4,35,CHAR_TO_LABEL('ols_pol','C:AD:BT4211'))
 SELECT * FROM dual;




In [73]:
school_fee_num = student_num
sf_slevel = SL_S
school_fee_student_id = [ student_id[i] for i in range(school_fee_num)]
school_fee_semster_id = [ i%semster_num+1 for i in range(school_fee_num)]
school_fee_school_fee = [ fake.random_number(digits=5) for i in range(school_fee_num)]
school_fee_school_fee_paid = [ "{0:.2f}".format(random.random()*school_fee_school_fee[i]) for i in range(school_fee_num)]
sf_sl = [get_security_label(sf_slevel, [FN], [school_label[i//2]]) for i in range(school_fee_num)]

school_fee_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO school_fee(student_id, semester_id, school_fee, school_fee_paid, {})\
    VALUES({},{},{},{},{})'.format(
    ols,
    wrap_single_quote(school_fee_student_id[i]),
    str(school_fee_semster_id[i]),
    str(school_fee_school_fee[i]),
    str(school_fee_school_fee_paid[i]),
    sf_sl[i]) for i in range(school_fee_num)]) + '\n SELECT * FROM dual;\n\n'

print(school_fee_insert_sql)

INSERT ALL
INTO school_fee(student_id, semester_id, school_fee, school_fee_paid, ols_pol_col)    VALUES('68abf025-9639-1a19-917b-3b04139be79e',1,51915,51566.96,CHAR_TO_LABEL('ols_pol','S:FN:SOC'))
INTO school_fee(student_id, semester_id, school_fee, school_fee_paid, ols_pol_col)    VALUES('0bf30fe0-4474-563f-7622-b7d0a8d2389f',2,40924,34300.73,CHAR_TO_LABEL('ols_pol','S:FN:SOC'))
INTO school_fee(student_id, semester_id, school_fee, school_fee_paid, ols_pol_col)    VALUES('aa92e04a-b38b-c6c8-e614-7fd9b23b749a',3,72663,22598.19,CHAR_TO_LABEL('ols_pol','S:FN:BIZ'))
INTO school_fee(student_id, semester_id, school_fee, school_fee_paid, ols_pol_col)    VALUES('cb832cc4-32c5-469a-46ea-625b4adbb0ba',4,9615,9411.87,CHAR_TO_LABEL('ols_pol','S:FN:BIZ'))
 SELECT * FROM dual;




In [74]:
staff_num = professor_num + admin_officer_num
staff_slevel = SL_HS
staff_id = professor_id + admin_officer_id
salary = [fake.random_number(digits=5) for i in range(staff_num)]
staff_sl = [get_security_label(staff_slevel,[], []) for i in range(staff_num)]
staff_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO staff_salary(staff_id, salary, {})\
    VALUES({},{},{})'.format(
    ols,
    wrap_single_quote(staff_id[i]),
    str(salary[i]),
    staff_sl[i]) for i in range(staff_num)]) + '\n SELECT * FROM dual;\n\n'
print(staff_insert_sql)

INSERT ALL
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('a456c770-e067-b7db-3b40-7788112c8ab4',96996,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('2b033463-3041-6f74-3a2c-28308f35b7a4',19033,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('aaadfe54-df73-799a-4d0e-469c8ff950b2',34093,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('d929fffd-cdc3-834d-f083-d89a60723793',17010,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('4a336a0f-5eca-1de9-63ae-31d861a5b333',96723,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('693749e8-5c17-97f5-e2f5-b374a2627257',1400,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('44a3e442-0e9b-e362-7fed-5fd1d3d2d496',59174,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary

In [75]:
with open('insert_data.sql', 'w') as f:
    f.write(school_insert_sql)

In [76]:
with open('insert_data.sql', 'a') as f:
    f.write(academic_insert_sql)

In [77]:
with open('insert_data.sql', 'a') as f:
    f.write(semster_insert_sql)

In [78]:
with open('insert_data.sql', 'a') as f:
    f.write(admin_officer_insert_sql)

In [79]:
with open('insert_data.sql', 'a') as f:
    f.write(professor_insert_sql)

In [80]:
with open('insert_data.sql', 'a') as f:
    f.write(course_insert_sql)

In [81]:
with open('insert_data.sql', 'a') as f:
    f.write(course_schedule_insert_sql)

In [82]:
with open('insert_data.sql', 'a') as f:
    f.write(student_insert_sql)

In [83]:
with open('insert_data.sql', 'a') as f:
    f.write(course_enrollment_insert_sql)

In [84]:
with open('insert_data.sql', 'a') as f:
    f.write(school_fee_insert_sql)

In [85]:
with open('insert_data.sql', 'a') as f:
    f.write(loan_insert_sql)

In [86]:
with open('insert_data.sql', 'a') as f:
    f.write(grade_insert_sql)

In [87]:
with open('insert_data.sql', 'a') as f:
    f.write(staff_insert_sql)